In [1]:
using Pkg
Pkg.activate("presentation_ebs")
Pkg.add(["BenchmarkTools", "MATLAB", "LoopVectorization",
         "MarketData", "TimeSeries", "Distributions",
         "KernelDensity", "StatsPlots", "ProgressMeter",
         "PyCall", "Conda", "ARCHModels"])
using Conda
Conda.add("arch-py", channel="conda-forge")
Pkg.update()


  Activating environment at `C:\Users\sabro\Documents\julia\notebooks\presentation\presentation_ebs\Project.toml`
    Updating registry at `C:\Users\sabro\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\sabro\Documents\julia\notebooks\presentation\presentation_ebs\Project.toml`
  No Changes to `C:\Users\sabro\Documents\julia\notebooks\presentation\presentation_ebs\Manifest.toml`
┌ Info: Running `conda install -y -c conda-forge arch-py` in root environment
└ @ Conda C:\Users\sabro\.julia\packages\Conda\sNGum\src\Conda.jl:128


Solving environment: ...working... done

# All requested packages already installed.



    Updating registry at `C:\Users\sabro\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
  No Changes to `C:\Users\sabro\Documents\julia\notebooks\presentation\presentation_ebs\Project.toml`
  No Changes to `C:\Users\sabro\Documents\julia\notebooks\presentation\presentation_ebs\Manifest.toml`


##### <p style="text-align: center; font-size: 300%"> The Unreasonable Effectiveness of Julia </p>
<p style="text-align: center; font-size: 200%"> Simon A. Broda </p>
<p style="text-align: center; font-size: 100%"> University of Amsterdam and Hochschule Luzern<br>
<a href="mailto:simon.broda@uzh.ch">s.a.broda@uva.nl</a> </p>
<img src="EULOGO.jpg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 20%;">
<p style="text-align: center; font-size: 100%"> This project has received funding from the European Research Council (ERC) under the European Union's Horizon 2020 research and innovation program (grant agreement No. 750559). </p>

# Outline
* The Julia Language
* Refresher on ARCH Models
* The `ARCHModels.jl` Package
   * Usage
   * Benchmarks


These slides are available at https://github.com/s-broda/presentation.

# The Julia Language
## General Information
* New programming language started at MIT.
* Designed with scientific computing in mind.
* Version 1.0 released in August 2018 after 9 years of development. Current version is 1.6.
* Free and open source software. Available at https://julialang.org/.
* In the words of its creators,
> We want a language that’s open source, with a liberal license. We want the speed of C with the dynamism of Ruby. We want a
language that’s homoiconic, with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want
something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as
powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn,
yet keeps the most serious hackers happy. We want it interactive and we want it compiled.



## Highlights for Scientists
* Interactive REPL (like Matlab, Python, R, etc.) allows for exploratory analysis, rapid prototyping.
* Unlike these, Julia is JIT compiled, hence fast (typically within a few percent of C)
* Syntax superficially similar to Matlab.
* Rich type system, multiple dispatch.
* Fast-growing eco-system with many state-of-the-art packages (e.g., `ForwardDiff.jl`, `DifferentialEquations.jl`, `JuMP`, ...).


## A Small Taste of Julia

* Let's say we want to implement a function that sums an array:

In [2]:
function mysum(x)
    s = zero(eltype(x))
    for i in eachindex(x)
        s += x[i]
    end
    s
end
mysum([1., 2., 3.])

6.0

* Let's benchmark it:

In [3]:
x = randn(10^7);

In [4]:
using BenchmarkTools
@btime mysum(x); # @btime runs the code a large number of times

  7.458 ms (1 allocation: 16 bytes)


* For comparison, the built-in function:

In [5]:
@btime sum(x);

  2.680 ms (1 allocation: 16 bytes)


* Close, but we are 2-3x slower.
* But we can do better! 2nd attempt:

In [51]:
function mysum(x)
    s = zero(eltype(x))
    @inbounds @simd for i in eachindex(x) # @simd exploits 'single instruction multiple data', a type of parallelism
        s += x[i]
    end
    s
end
mysum([1., 2., 3.])

6.0

In [52]:
@btime mysum(x);

  2.638 ms (1 allocation: 16 bytes)


* Not bad: by just adding a simple decorator, we now match the speed of the built-in function!
* This is, in fact, expected: the built-in function is implemented in Julia, like most of the standard library.
* We can even look at the code:

In [53]:
@which sum(x)

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:873

## One more thing...

In [9]:
using LoopVectorization
function mysum(x)
    s = zero(eltype(x))
    @tturbo for i in eachindex(x)
        s += x[i]
    end
    s
end
mysum([1., 2., 3.])

6.0

In [10]:
@btime mysum(x);

  948.600 μs (1 allocation: 16 bytes)


* __Wow :o__
* This difference comes from multi-threading; @tturbo exploits all the cores the machine has available.

# Refresher on ARCH Models
* Daily financial returns data exhibit a number of *stylized facts*:
  * Volatility clustering
  * Non-Gaussianity, fat tails
  * Leverage effects: negative returns increase future volatility
* Other types of data (e.g., changes in interest rates) exhibit similar phenomena.
* These effects are important in many areas in finance, in particular in risk management.
* [G]ARCH ([**G**eneralized] **A**utoregressive **C**onditional **V**olatility) models are the most popular for modelling them.

## Example: volatility clustering in AAPL returns
* The `MarketData` package allows us to download historical data from Yahoo Finance. 

In [11]:
using MarketData, TimeSeries
r = percentchange(yahoo(:AAPL)[:AdjClose]) # returns a TimeArray
data = values(r) # an array containing just the plain data

if !isfile("returns.svg") || !isfile("kde.svg")
    using Distributions, KernelDensity, StatsPlots
    plot(r, title="Volatility Clustering", legend=:none, ylabel="\$r_t\$")
    savefig("returns.svg")
    plot(kde(data), label="Kernel Density", title="Fat Tails")
    plot!(fit(Normal, data), label="Fitted Normal")
    savefig("kde.svg")
end

<img src="returns.svg" alt="RETURNS" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

<img src="kde.svg" alt="Kernel Density" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">



## (G)ARCH Models
* Basic setup: given a sample of financial returns $\{r_t\}_{t\in\{1,\ldots,T\}}$, decompose $r_t$ as
$$
r_t=\mu_t+\sigma_tz_t, \quad z_t\stackrel{i.i.d.}{\sim}(0,1),
$$
where $\mu_t\equiv\mathbb{E}[r_t\mid \mathcal{F}_{t-1}]$ and $\sigma_t^2\equiv \mathbb{E}[(r_t-\mu_t)^2\mid \mathcal{F}_{t-1}]$.
* Assume $\mu_t=0$ for simplicity. Focus is on the *volatility* $\sigma_t$. G(ARCH) models make $\sigma_t$ a function of *past* returns and variances. Examples: 

## Examples
* ARCH(q) (Engle, Econometrica 1982):
$$\sigma_t^2=\omega+\sum_{i=1}^q \alpha_ir_{t-i}^2%,\quad \omega,\alpha_i>0,\quad \sum_{i=1}^q\alpha_i<1.
$$
* GARCH(p, q) (Bollerslev, JoE 1986)
$$\sigma_t^2=\omega+ \sum_{i=1}^p\beta_{i}\sigma_{t-i}^2 + \sum_{i=1}^q\alpha_ir_{t-i}^2%,\quad \omega,\alpha_i,\beta_i>0,\quad \sum_{i=1}^{\max p,q} \alpha_i+\beta_i<1.
$$
* EGARCH(o, p, q) (Nelson, Econometrica 1991)
$$\log(\sigma_t^2)=\omega+\sum_{i=1}^o\gamma_{i}z_{t-i}+\sum_{i=1}^p\beta_i\log(\sigma_{t-i}^2)+\sum_{i=1}^q \alpha_i (|z_t|-\mathbb{E}|z_t|)%, \quad \sum_{i=1}^p \beta_i<0.
$$
* and many (many, many...) others 
  



## Estimation
* G(ARCH) models are usually estimated by maximum likelihood: with $f_z$ denoting the density of $z_t$,
$$\max \sum_t \log f(r_t\mid \mathcal{F}_{t-1})=\max \sum_t \log f_z(r_t/\sigma_t)-\log \sigma_t.$$
* Recursive nature of $\sigma_t$ means the computation cannot be "vectorized" $\Rightarrow$ loops.
* Julia is very well suited for this. Matlab (and the `rugarch` package for Python) have to implement the likelihood in C.



# The `ARCHModels.jl` Package
## Installation
* `ARCHModels.jl` is a registered julia package. To install it, just do `using Pkg; Pkg.add("ARCHModels")`
*  Extensive documentation available at https://s-broda.github.io/ARCHModels.jl/stable/.

## Key Features
* Supports simulating, estimating, forecasting, and backtesting ARCH models.
* Currently: ARCH, GARCH, TGARCH, and EGARCH models of arbitrary orders, with Gaussian, Student's $t$, and GED errors.
* Regression and ARMA models for the mean equation.
* CCC and DCC models for multivariate returns.
* Entirely written in Julia.
* Designed to be easily extensible with new models and distributions (nice thesis project...)
* Gradients and Hessians (for both numerical maximization of the likelihood and constructing standard errors) are obtained by [automatic differentiation](http://www.autodiff.org/?module=Introduction) via `ForwardDiff.jl`.

## Usage

* The first step in building an ARCH model is usually to test for the presence of volatility clustering.
* `ARCHModels.jl` provides [Engle's (Econometrica 1982)](https://doi.org/10.2307/1912773) ARCH-LM test for this.
* The null is that $\gamma_i=0$ in the auxiliary regression
$$
r_t^2=\alpha+\gamma_1 r_{t-1}^2+\gamma_2 r_{t-2}^2+\cdots+\gamma_p r_{t-p}^2+\epsilon_t.
$$

In [54]:
using ARCHModels
ARCHLMTest(data, 4) # p = 4 lags

ARCH LM test for conditional heteroskedasticity
-----------------------------------------------
Population details:
    parameter of interest:   T⋅R² in auxiliary regression
    value under h_0:         0
    point estimate:          147.398

Test summary:
    outcome with 95% confidence: reject h_0
    p-value:                     <1e-30

Details:
    sample size:                    10205
    number of lags:                 4
    LM statistic:                   147.398


* Unsurprisingly, the test rejects.

* Fitting a GARCH model is as simple as

In [13]:
model = fit(GARCH{1, 1}, data)


GARCH{1, 1} model with Gaussian errors, T=10205.

Mean equation parameters:
─────────────────────────────────────────────
     Estimate    Std.Error  z value  Pr(>|z|)
─────────────────────────────────────────────
μ  0.00184929  0.000257205  7.18994    <1e-12
─────────────────────────────────────────────

Volatility parameters:
─────────────────────────────────────────────
      Estimate  Std.Error   z value  Pr(>|z|)
─────────────────────────────────────────────
ω   5.67925e-6  2.9651e-6   1.91537    0.0554
β₁  0.929851    0.0136741  68.0008     <1e-99
α₁  0.0674646   0.0176432   3.82384    0.0001
─────────────────────────────────────────────


* Alternatively, the (multithreaded!) `selectmodel` method does automatic model selection (i.e., chooses $p$ and $q$ by minimizing the AIC/AICC/BICC).

In [14]:
model2 = selectmodel(GARCH, data; maxlags=3, criterion=bic) # optional keyword arguments


GARCH{3, 1} model with Gaussian errors, T=10205.

Mean equation parameters:
─────────────────────────────────────────────
     Estimate    Std.Error  z value  Pr(>|z|)
─────────────────────────────────────────────
μ  0.00201148  0.000241287  8.33647    <1e-16
─────────────────────────────────────────────

Volatility parameters:
──────────────────────────────────────────────────
       Estimate   Std.Error      z value  Pr(>|z|)
──────────────────────────────────────────────────
ω   1.34075e-5   1.42247e-6  9.42551        <1e-20
β₁  0.520858     0.088458    5.88819        <1e-08
β₂  1.36846e-48  3.42138e-7  3.99972e-42    1.0000
β₃  0.358345     0.0886664   4.0415         <1e-04
α₁  0.101772     0.0134813   7.54909        <1e-13
──────────────────────────────────────────────────


* The return value is of type `UnivariateARCHModel`:

In [15]:
typeof(model)

UnivariateARCHModel{Float64, GARCH{1, 1, Float64}, StdNormal{Float64}, Intercept{Float64}}

* `UnivariateARCHModel` supports many useful methods, such as 
 `confint`, `aic`, `bic`, `aicc`, `informationmatrix`, `score`, `vcov`:


In [16]:
aic(model)

-45895.13689151763

* A `UnivariateARCHModel` essentially consists of a volatility specification (of type `UnivariateVolatilitySpec`),
an error distribution (of type `StandardizedDistribution`), and a mean specification (of type `MeanSpec`).
* The following are currently implemented:

In [17]:
print.(string.(subtypes(UnivariateVolatilitySpec)) .* " ");

EGARCH TGARCH 

In [18]:
print.(string.(subtypes(StandardizedDistribution))[2:end] .* " ");

StdGED StdNormal StdSkewT StdT 

In [19]:
print.(string.(subtypes(MeanSpec)).* " ");

ARMA Intercept NoIntercept Regression 

* We can use these to construct an ARCHModel manually:

In [20]:
T = 10^4
mydata = zeros(T)
volaspec = EGARCH{1, 1, 1}([0.02, .09, .83, .01])
using Random; Random.seed!(1)
mymodel = UnivariateARCHModel(volaspec, mydata; dist=StdT(4.)) # dist is an optional keyword argument


EGARCH{1, 1, 1} model with Student's t errors, T=10000.


─────────────────────────────────────────────────
                              ω    γ₁    β₁    α₁
─────────────────────────────────────────────────
Volatility parameters:     0.02  0.09  0.83  0.01
─────────────────────────────────────────────────
──────────────────────────────
                             ν
──────────────────────────────
Distribution parameters:   4.0
──────────────────────────────


* With a `UnivariateARCHModel` in hand, we can do things like

In [21]:
simulate!(mymodel) # by convention, the bang indicates that the method modifies its argument


EGARCH{1, 1, 1} model with Student's t errors, T=10000.


─────────────────────────────────────────────────
                              ω    γ₁    β₁    α₁
─────────────────────────────────────────────────
Volatility parameters:     0.02  0.09  0.83  0.01
─────────────────────────────────────────────────
──────────────────────────────
                             ν
──────────────────────────────
Distribution parameters:   4.0
──────────────────────────────


In [22]:
fit!(mymodel)


EGARCH{1, 1, 1} model with Student's t errors, T=10000.


Volatility parameters:
───────────────────────────────────────────────
       Estimate   Std.Error   z value  Pr(>|z|)
───────────────────────────────────────────────
ω    0.0274054   0.00733817   3.73464    0.0002
γ₁   0.113959    0.0131327    8.67752    <1e-17
β₁   0.811765    0.0244774   33.1638     <1e-99
α₁  -0.00637894  0.0153358   -0.41595    0.6774
───────────────────────────────────────────────

Distribution parameters:
─────────────────────────────────────────
   Estimate  Std.Error  z value  Pr(>|z|)
─────────────────────────────────────────
ν   3.83768   0.152659   25.139    <1e-99
─────────────────────────────────────────


In [23]:
ARCHLMTest(mymodel) # tests the standardized residuals

ARCH LM test for conditional heteroskedasticity
-----------------------------------------------
Population details:
    parameter of interest:   T⋅R² in auxiliary regression
    value under h_0:         0
    point estimate:          0.0873996

Test summary:
    outcome with 95% confidence: fail to reject h_0
    p-value:                     0.7675

Details:
    sample size:                    10000
    number of lags:                 1
    LM statistic:                   0.0873996


In [24]:
predict(mymodel, :volatility) # (:volatility | :variance | :return | :VaR)

0.9582988646322497

## Value at Risk

* In-sample Value-at-Risk estimates are available via the `VaRs` function

In [25]:
model = fit(GARCH{1, 1}, data)
vars = VaRs(model, 0.05)

if !isfile("VaRplot.svg")
    using Plots
    plot(-data, legend=:none, xlabel="\$t\$", ylabel="\$-r_t\$", title="Value at Risk, In-Sample")
    plot!(vars, color=:purple)
    savefig("VaRplot.svg"); 
end

<img src="VaRplot.svg" alt="VALUE AT RISK" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

* How about out-of-sample (backtesting)?
* Requires re-estimating the model at each time step.
* Not built in, but easy to do:

In [26]:
using ProgressMeter

T = length(data)
windowsize = 1000
vars = similar(data); fill!(vars, NaN)
@showprogress "Fitting $(T-1-windowsize) GARCH models: " for t = windowsize+1:T-1
    m = fit(GARCH{1, 1}, data[t-windowsize:t])
    vars[t+1] = predict(m, :VaR; level=0.05)    
end

if !isfile("VaRplot_oos.svg")
    using Plots
    plot(-data, legend=:none, xlabel="\$t\$", ylabel="\$-r_t\$", title="Value at Risk, Out-of-Sample")
    plot!(vars, color=:purple)
    savefig("VaRplot_oos.svg"); 
end

Fitting 9204 GARCH models: 100%|████████████████████████| Time: 0:00:24


<img src="VaRplot_oos.svg" alt="VALUE AT RISK OUT OF SAMPLE" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

* We can backtest these out-of-sample VaR predictions using Engle and Manganelli's (2004) dynamic quantile test.
* The test is based on a *hit series* 
$$I_t\equiv\cases{1, r_t<-VaR_t\\0, \mbox{otherwise.}}$$
* The null is that $\alpha=\beta=\gamma=0$ in the auxiliary regression
$$
I_t-0.05 = \alpha +\beta I_{t-1} + \gamma VaR_t+\epsilon_t.
$$

In [27]:
DQTest(data[windowsize+1:end], vars[windowsize+1:end], 0.05)

Engle and Manganelli's (2004) DQ test (out of sample)
-----------------------------------------------------
Population details:
    parameter of interest:   Wald statistic in auxiliary regression
    value under h_0:         0
    point estimate:          35.834

Test summary:
    outcome with 95% confidence: reject h_0
    p-value:                     <1e-07

Details:
    sample size:                    9205
    number of lags:                 1
    VaR level:                      0.05
    DQ statistic:                   35.834


# Benchmarks
  * Bollerslev and Ghysels (JBES 1996) data is de facto standard in comparing implementations of GARCH models.
  * Data consist of daily German mark/British pound exchange rates (1974 observations).
  * Available in `ARCHModels.jl` as the constant `BG96`.

In [28]:
if !isfile("DMGBP.svg")
    using Plots
    plot(BG96, legend=:none, title="Bollerslev and Ghysels (1996) Data", ylabel="\$r_t\$", xlabel="\$t\$")
    savefig("DMGBP.svg")
end

<img src="DMGBP.svg" alt="Bollerslev and Ghysels (1996) data" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

## GARCH
* Fitting in Julia:

In [29]:
using BenchmarkTools
@btime fit(GARCH{1, 1}, $BG96)

  3.208 ms (2546 allocations: 203.33 KiB)



GARCH{1, 1} model with Gaussian errors, T=1974.

Mean equation parameters:
───────────────────────────────────────────────
      Estimate   Std.Error    z value  Pr(>|z|)
───────────────────────────────────────────────
μ  -0.00616637  0.00920152  -0.670147    0.5028
───────────────────────────────────────────────

Volatility parameters:
─────────────────────────────────────────────
     Estimate   Std.Error   z value  Pr(>|z|)
─────────────────────────────────────────────
ω   0.0107606  0.00649303   1.65725    0.0975
β₁  0.805875   0.0724765   11.1191     <1e-27
α₁  0.153411   0.0536404    2.86       0.0042
─────────────────────────────────────────────


* Now Matlab:

In [36]:
using MATLAB # this is for calling Matlab from within Julia. I do this because a Jupyter notebook
mat"version" # only supports a single language at a time.

"9.10.0.1669831 (R2021a) Update 2"

In [37]:
mat"""
estimate(garch('ARCHLags', 1, 'GARCHLags', 1, 'offset', NaN), $BG96); 0;
f = @()estimate(garch('ARCHLags', 1, 'GARCHLags', 1, 'offset', NaN), $BG96, 'Display', 'off');
t = timeit(f);
disp(['Time per run: ', num2str(t * 1000), ' ms'])
"""

 
    GARCH(1,1) Conditional Variance Model with Offset (Gaussian Distribution):
 
                  Value       StandardError    TStatistic      PValue  
                __________    _____________    __________    __________

    Constant      0.010761       0.001323         8.1342     4.1454e-16
    GARCH{1}       0.80597        0.01656         48.669              0
    ARCH{1}        0.15313       0.013974         10.959     6.0379e-28
    Offset      -0.0061904      0.0084336       -0.73402        0.46294

Time per run: 33.029 ms


0.0

* ARCH.jl is faster by a factor of about 10 or more, depending on the machine, despite Matlab calling into compiled C code.
* Estimates are quite similar, but standard errors and $t$-statistics differ.
* So which standard errors are correct? Let's compare with the results from Brooks et. al. (Int. J. Fcst. 2001).
* Brooks et. al. compare implementations of the GARCH(1, 1) model.
* They give the following estimates (**$t$-stats**) as benchmarks:
$\mu=−0.00619$ $(\mathbf{−0.67})$, $\omega=0.0108$ $(\mathbf{1.66})$, $\beta_1=0.806$ $(\mathbf{11.11})$, $\alpha_1=0.153$ $(\mathbf{2.86})$.
* `ARCHModels.jl` is bang on. Not sure what Matlab is doing. Look at those t-stats!


* Finally, Python:

In [45]:
using PyCall # for calling Python from within Julia
py"""import pandas as pd
import timeit
import functools
from arch.univariate.mean import ConstantMean
from arch.univariate.distribution import Normal
from arch.univariate import GARCH
am = ConstantMean($BG96, rescale=False)
am.volatility = GARCH(1, 0, 1)
N = 100
print(am.fit(disp="off"))
print("Time per run:", timeit.timeit(functools.partial(am.fit, disp="off"), number=N) / N * 1000, "ms") """

                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      y   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1104.52
Distribution:                  Normal   AIC:                           2217.04
Method:            Maximum Likelihood   BIC:                           2239.39
                                        No. Observations:                 1974
Date:                Sun, Jun 06 2021   Df Residuals:                     1973
Time:                        18:42:58   Df Model:                            1
                                  Mean Model                                  
                  coef    std err          t      P>|t|       95.0% Conf. Int.
------------------------------------------------------------------------------
mu         -6.0765e-03  9.197e-03     -0.661      0.

# Conclusion

* We beat Matlab by a factor of about 10 and Python's `arch` package by a factor of about 6, all while delivering estimates that are closer to the benchmark. We also beat R's `rugarch` package by a factor of about 30.
* This is despite the fact that all these packages implement the numerically heavy stuff in C rather than natively.
* This is the _unreasonable effectiveness of Julia_: it walks like Python and runs like C.

#  TODO
* More distributions (NIG, $\alpha$-Stable, ...)
* More GARCH models (APARCH, RiskMetrics, IGARCH, ...)
* Portfolio VaR: Compute the VaR of a portfolio, possibly containing derivatives, via the multivariate Delta-Gamma approximation.
* Made feasible by the results of Broda and Arismendi (Biometrika 2021), under the assumption of a multivariate generalize hyperbolic (MGHyp) distribution.
* This flexible distribution nests, among others, the Normal, Student's $t$, Variance Gamma, Normal Inverse Gaussian, ...
* The scary looking expression on the next slide is already available with my Julia package `QuadraticFormsMGHyp.jl`.



__Theorem__ (Broda and Arismendi, Biometrika 2021) Let $L= a_0+{a}^{T}X+X^{T}{A}X$, where $X\sim \mathrm{MGHyp({{\mu},{C},{\gamma},\lambda,\chi,\psi})}$. Then
\begin{equation}\label{eq:mghcdf}
\mathbb{P}(L\leq l)=\frac{1}{2}-\frac{1}{\pi}\int_{0}^{\infty}\mathrm{Im}\left\{
\Xi_{\lambda}(s,-xs,\chi,\psi)\right\}\frac{\mathrm{d}s}{s},
\end{equation}
where $x= l-a_0-{a}^{T}{\mu}-{\mu}^{T}{A}{\mu}$,
\begin{gather*}
\Xi_{\lambda}(s,t,\chi,\psi)=\frac{k_\lambda\{\chi-2\alpha_2(s)-2\mathrm{i}t,\psi-2\alpha_1(s)\}}{k_\lambda(\chi,\psi)}\rho(s),\\
\alpha_1(s)=\mathrm{i}ks-\frac{1}{2}s^2\sum_{j=1}^d\frac{e_j^2}{1-2\mathrm{i}s\lambda_j},\quad
\alpha_2(s)=-\frac{1}{2}s^2\sum_{j=1}^d\frac{d_j^2}{1-2\mathrm{i}s\lambda_j},\\
\rho(s)=\exp\left(\mathrm{i}sc-s^2 \sum_{j=1}^d\frac{d_je_j}{1-2\mathrm{i}s\lambda_j}\right)\prod_{j=1}^d(1-2\mathrm{i}s\lambda_j)^{-1/2},
\end{gather*}
$\lambda_{j}, j \in\{1,\ldots,d\}$ are the eigenvalues of ${C}^{T}{A}{C}$, ${P}$ is an orthogonal matrix with the corresponding eigenvectors as its columns,
$c= {a}^{T}{\gamma}+ 2 {\mu}^{T}{A}{\gamma}$, $d_j$ and $e_j$ are the individual elements of ${d}={a}^{T}{CP}+2{\mu}^{T}{ACP}$ and ${e}= {\gamma}{ACP}$, and $k={\gamma}^{T}{A}{\gamma}$.


# References
* Bollerslev, T (1986). Generalized autoregressive conditional heteroskedasticity. *Journal of Econometrics* **31**, 307–327.
* Bollerslev, T. & Ghysels, E. (1996). Periodic Autoregressive Conditional Heteroscedasticity. *Journal of Business & Economic Statistics* **14**, 139-151. https://doi.org/10.1080/07350015.1996.10524640.
* Broda, S.A. & Arismendi Zambrano, J. (2021). On quadratic forms in multivariate generalized hyperbolic
random vectors. *Biometrika* **108**, https://doi.org/413-424.0.1093/biomet/asaa067.
* __Broda, S.A. & Paolella, M. (2020). `ARCHModels.jl`: Estimating Arch Models in Julia. Available on SSRN. https://dx.doi.org/10.2139/ssrn.3551503__
* Brooks, C., Burke, S. P., & Persand, G. (2001). Benchmarks and the accuracy of GARCH model estimation. *International Journal of Forecasting* **17**, 45-56. https://doi.org/10.1016/S0169-2070(00)00070-4.
* Engle, R. F. (1982). Autoregressive Conditional Heteroscedasticity with Estimates of the Variance of United Kingdom Inflation. *Econometrica* **50**, 987-1007. https://doi.org/10.2307/1912773.
* Engle, R. F., and Manganelli, S. (2004). CAViaR: Conditional Autoregressive Value at Risk by Regression Quantiles. *Journal of Business & Economic Statistics* **22**, 367-381. https://doi.org/10.1198/073500104000000370
* Nelson, D.B. (1991). Conditional Heteroskedasticity in Asset Returns: A New Approach. *Econometrica* **59**, 347--370. https://doi.org/10.2307/2938260.



##### <p style="text-align: center; font-size: 300%"> Thank you! Questions? </p>